# 1) Scraping des données

Tout d'abord, il nous a fallu récupérer les données utiles des hôtels parisiens sur le site Booking.com. Nous nous sommes rendus compte que ces informations n'étaient pas directement accessibles via le code source, mais qu'elles l'étaient avec la fonction "Inspecter l'élément" de nos navigateurs Internet. Cela est certainement dû à l'utilisation par le site de scripts Java.

Nous avons donc créé une fonction qui ouvre une page Web grâce à Selenium, puis qui cherche sur cette page l'ensemble des entrées correspondant aux classes contenant les informations utiles. Ces entrées sont sauvegardées dans des tableaux. Une fois les données recueillies, le navigateur se ferme.

In [3]:
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import numpy as np

Nous souhaitons utiliser cette fonction sur les différentes pages du site afin de disposer d'une base de données suffisamment importante. Nous nous sommes rendus compte que la page $i+1$ est accessible en ajoutant "&offset25$(i+1)$" à l'URL de la page. De fait, nous avons créé une liste avec les 40 premières URL (de manière à obtenir une base avec 1 000 hôtels).

In [4]:
def cree_liste(url):
    liste=[]
    for i in range(40):
        if i==0:
            liste.append(url)
        else:
            liste.append(url+'&offset='+str(25*i))
    return(liste)

# 1.1. Web-scraping pour l'évolution des prix

Afin de pouvoir comparer les prix, il faut pouvoir changer de jour de réservation de façon automatique. Les jours (pour un mois donné) de réservation sont indiqués dans l'URL grâce à la référence "checkout_monthday".

In [5]:
jours = [(a,11) for a in range(16,31)]+[(a,12) for a in range(1,32)]
jours[:3]

[(16, 11), (17, 11), (18, 11)]

In [2]:
prix = []
noms_hotels = []
#prix = [] # En-dessous, l'URL de départ
# Pour chaque jour, on change l'URL puis on récupère les prix sur les 40 pages
for jour in range(len(jours)-1): # A chaque fois, on remet l'URL de départ. Cette solution n'est pas optimale en termes de temps de calcul, mais évite d'écrire des lignes supplémentaires car il faudrait s'adapter à l'URL qui est changée à chaque itération.
    #print(jour)
    url = 'https://www.booking.com/searchresults.fr.html?label=gog235jc-1FCAMoTTjjAkgNWANoTYgBAZgBDbgBF8gBDNgBAegBAfgBAogCAagCA7gCw9rQiwbAAgHSAiQ0NDlkNGU5ZC03NzAxLTQ5MzItYjdkYS0wMzVkNDI4NGRiNGXYAgXgAgE&sid=81cd22762f2169ba954d21a297ff0a64&aid=356980&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D356980%3Blabel%3Dgog235jc-1FCAMoTTjjAkgNWANoTYgBAZgBDbgBF8gBDNgBAegBAfgBAogCAagCA7gCw9rQiwbAAgHSAiQ0NDlkNGU5ZC03NzAxLTQ5MzItYjdkYS0wMzVkNDI4NGRiNGXYAgXgAgE%3Bsid%3D81cd22762f2169ba954d21a297ff0a64%3Btmpl%3Dsearchresults%3Bcity%3D-1456928%3Bclass_interval%3D1%3Bdest_id%3D-1456928%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Boffset%3D0%3Bpostcard%3D0%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrpvid%3D227a56f9e7060099%3Bss_all%3D0%3Bssb%3Dempty%3Bsshis%3D0%3Btop_ufis%3D1%26%3B&ss=Paris&is_ski_area=0&ssne=Paris&ssne_untouched=Paris&city=-1456928&checkin_year=2021&checkin_month=11&checkin_monthday=6&checkout_year=2021&checkout_month=11&checkout_monthday=7&group_adults=2&group_children=0&no_rooms=1&sb_changed_dates=1&from_sf=1&nflt=ht_id%3D204'
    url = url.replace('checkin_year=2021&checkin_month=11&checkin_monthday=6&checkout_year=2021&checkout_month=11&checkout_monthday=7','checkin_year=2021&checkin_month='+str(jours[jour][1])+'&checkin_monthday='+str(jours[jour][0])+'&checkout_year=2021&checkout_month='+str(jours[jour+1][1])+'&checkout_monthday='+str(jours[jour+1][0]))
    #print(url)
    liste = cree_liste(url)
    for i in range(4):
        driver = webdriver.Chrome()
        driver.get(liste[i])
        noms_hotels.append([hotel.text for hotel in wait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "fde444d7ef._c445487e2")))])
        prix.append([prix.text for prix in wait(driver,10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "fde444d7ef._e885fdc12")))])
        driver.quit()

print(prix, noms_hotels)

NameError: name 'cree_liste' is not defined

In [65]:
prix2 = {}
for x in range(16, 31):
    prix2["{0}/11/2021".format(x)] = prix[(x-16)*4]+prix[(x-16)*4 +1]+prix[(x-16)*4 +2]+prix[(x-16)*4 +3] #-1+(x-15)*4# prix[(x-16)*4:-1+(x-15)*4]
for x in range(1, 31):
    prix2["{0}/12/2021".format(x)] = prix[60+(x-1)*4]+prix[61+(x-1)*4]+prix[62+(x-1)*4]+prix[63+(x-1)*4]
print(prix2)

nom_hotel_2 = {}
for x in range(16, 31):
    nom_hotel_2["{0}/11/2021".format(x)] = noms_hotels[(x-16)*4]+noms_hotels[(x-16)*4 +1]+noms_hotels[(x-16)*4 +2]+noms_hotels[(x-16)*4 +3] #-1+(x-15)*4# prix[(x-16)*4:-1+(x-15)*4]
for x in range(1, 31):
    nom_hotel_2["{0}/12/2021".format(x)] = noms_hotels[60+(x-1)*4]+noms_hotels[61+(x-1)*4]+noms_hotels[62+(x-1)*4]+noms_hotels[63+(x-1)*4]
#nom_hotel_2
    
#for x in range(1,32):
#    prix2["prix{0}12".format(x)] = prix[60+(x-1)*4:45+]

{'16/11/2021': ['€ 226', '€ 132', '€ 196', '€ 146', '€ 126', '€ 150', '€ 174', '€ 193', '€ 143', '€ 194', '€ 153', '€ 183', '€ 195', '€ 206', '€ 225', '€ 169', '€ 215', '€ 185', '€ 225', '€ 215', '€ 278', '€ 129', '€ 272', '€ 179', '€ 438', '€ 186', '€ 194', '€ 265', '€ 357', '€ 245', '€ 214', '€ 286', '€ 245', '€ 205', '€ 166,76', '€ 263', '€ 258', '€ 205', '€ 366', '€ 403', '€ 417', '€ 284,78', '€ 344', '€ 300', '€ 276', '€ 235', '€ 147', '€ 336', '€ 253', '€ 228', '€ 233', '€ 415', '€ 118', '€ 81', '€ 254', '€ 185', '€ 69', '€ 176', '€ 171', '€ 49', '€ 102', '€ 107', '€ 115', '€ 144', '€ 132', '€ 109', '€ 154', '€ 133', '€ 190', '€ 117', '€ 191', '€ 109', '€ 143', '€ 225', '€ 107', '€ 149', '€ 249', '€ 123', '€ 129', '€ 165', '€ 147', '€ 87', '€ 231', '€ 116', '€ 182', '€ 166', '€ 153', '€ 149', '€ 123', '€ 171', '€ 130', '€ 123', '€ 131', '€ 169', '€ 181', '€ 165', '€ 181', '€ 89', '€ 129', '€ 158'], '17/11/2021': ['€ 137', '€ 128', '€ 132', '€ 196', '€ 184', '€ 123', '€ 158', '€ 1

In [66]:
testdfprix = pd.DataFrame.from_dict(prix2)
testdfprix

,16/11/2021,17/11/2021,18/11/2021,19/11/2021,20/11/2021,21/11/2021,22/11/2021,23/11/2021,24/11/2021,25/11/2021,...,21/12/2021,22/12/2021,23/12/2021,24/12/2021,25/12/2021,26/12/2021,27/12/2021,28/12/2021,29/12/2021,30/12/2021
0,€ 226,€ 137,€ 122,€ 93,€ 190,€ 93,€ 118,€ 141,€ 141,€ 141,...,€ 136,€ 136,€ 136,€ 114,€ 114,€ 114,€ 136,€ 136,€ 275,€ 147
1,€ 132,€ 128,€ 128,€ 173,€ 107,€ 177,€ 101,€ 107,€ 107,€ 121,...,€ 120,€ 148,€ 111,€ 120,€ 124,€ 179,€ 157,€ 157,€ 147,€ 147
2,€ 196,€ 132,€ 157,€ 99,€ 173,€ 87,€ 117,€ 123,€ 157,€ 132,...,€ 127,€ 115,€ 124,€ 123,€ 120,€ 88,€ 125,€ 125,€ 194,€ 225
3,€ 146,€ 196,€ 117,€ 136,€ 157,€ 85,€ 148,€ 157,€ 132,€ 148,...,€ 142,€ 128,€ 123,€ 133,€ 123,€ 115,€ 160,€ 183,€ 174,€ 166
4,€ 126,€ 184,€ 165,€ 148,€ 174,€ 138,€ 147,€ 179,€ 179,€ 179,...,€ 148,€ 143,€ 115,€ 115,€ 115,€ 108,€ 146,€ 146,€ 165,€ 181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,€ 165,€ 91,€ 152,€ 103,€ 98,€ 63,€ 107,€ 139,€ 165,€ 104,...,€ 163,€ 89,€ 147,€ 159,€ 119,€ 88,€ 105,"€ 132,76",€ 119,€ 149
96,€ 181,€ 140,€ 89,€ 143,€ 112,€ 168,€ 134,€ 152,€ 104,€ 133,...,€ 83,€ 103,€ 94,€ 119,"€ 113,98",€ 93,€ 150,€ 156,€ 177,€ 180
97,€ 89,€ 157,€ 231,€ 166,€ 107,€ 62,€ 87,€ 97,€ 148,€ 135,...,€ 156,€ 93,€ 76,€ 84,€ 99,€ 89,€ 165,€ 108,€ 139,€ 180
98,€ 129,€ 191,€ 89,€ 165,€ 137,€ 126,€ 104,€ 152,€ 185,€ 195,...,€ 104,€ 103,€ 119,€ 123,€ 159,€ 114,€ 103,€ 183,€ 143,€ 190


In [67]:
testdfhotel = pd.DataFrame.from_dict(nom_hotel_2)
testdfhotel

,16/11/2021,17/11/2021,18/11/2021,19/11/2021,20/11/2021,21/11/2021,22/11/2021,23/11/2021,24/11/2021,25/11/2021,...,21/12/2021,22/12/2021,23/12/2021,24/12/2021,25/12/2021,26/12/2021,27/12/2021,28/12/2021,29/12/2021,30/12/2021
0,Room Mate Alain - Champs-Elysées,Hotel Ariane Montparnasse by Patrick Hayat,Hotel Ariane Montparnasse by Patrick Hayat,Hotel Ariane Montparnasse by Patrick Hayat,Room Mate Alain - Champs-Elysées,Hotel Ariane Montparnasse by Patrick Hayat,Hotel Ariane Montparnasse by Patrick Hayat,Hotel Ariane Montparnasse by Patrick Hayat,Hotel Ariane Montparnasse by Patrick Hayat,Hotel Ariane Montparnasse by Patrick Hayat,...,Hotel Ariane Montparnasse by Patrick Hayat,Hotel Ariane Montparnasse by Patrick Hayat,Hotel Ariane Montparnasse by Patrick Hayat,Hotel Ariane Montparnasse by Patrick Hayat,Hotel Ariane Montparnasse by Patrick Hayat,Hotel Ariane Montparnasse by Patrick Hayat,Hotel Ariane Montparnasse by Patrick Hayat,Hotel Ariane Montparnasse by Patrick Hayat,Room Mate Alain - Champs-Elysées,Hotel Ariane Montparnasse by Patrick Hayat
1,Absolute Hotel Paris République,Hotel de l'Aqueduc,Hotel de l'Aqueduc,TRIBE Paris Batignolles,The ReMIX Hotel,Hôtel l'Echiquier Opéra Paris - MGallery,Hotel de l'Aqueduc,Hotel de l'Aqueduc,Hotel de l'Aqueduc,Hotel de l'Aqueduc,...,Hotel de l'Aqueduc,TRIBE Paris Batignolles,Hotel de l'Aqueduc,Hotel de l'Aqueduc,Hotel de l'Aqueduc,Hôtel l'Echiquier Opéra Paris - MGallery,TRIBE Paris Batignolles,TRIBE Paris Batignolles,Trianon Gare de Lyon,Trianon Gare de Lyon
2,Hôtel & Spa Saint Jacques,Absolute Hotel Paris République,TRIBE Paris Batignolles,The ReMIX Hotel,TRIBE Paris Batignolles,Mary's Hotel République,Absolute Hotel Paris République,Absolute Hotel Paris République,TRIBE Paris Batignolles,Absolute Hotel Paris République,...,Absolute Hotel Paris République,The ReMIX Hotel,Trianon Gare de Lyon,Absolute Hotel Paris République,Trianon Gare de Lyon,Mary's Hotel République,The ReMIX Hotel,The ReMIX Hotel,TRIBE Paris Batignolles,TRIBE Paris Batignolles
3,Hotel de l'Aqueduc,Hôtel & Spa Saint Jacques,Absolute Hotel Paris République,Absolute Hotel Paris République,Hotel de l'Aqueduc,Hotel de l'Aqueduc,TRIBE Paris Batignolles,TRIBE Paris Batignolles,Absolute Hotel Paris République,TRIBE Paris Batignolles,...,Trianon Gare de Lyon,Hotel Aida Opera,Absolute Hotel Paris République,Trianon Gare de Lyon,Absolute Hotel Paris République,Hotel de l'Aqueduc,Hotel de l'Aqueduc,Hotel de l'Aqueduc,Hotel de l'Aqueduc,Hôtel Des Arts-Bastille
4,Hôtel Hor Les Lumières,TRIBE Paris Batignolles,Trianon Gare de Lyon,Hotel de l'Aqueduc,Trianon Gare de Lyon,Trianon Gare de Lyon,Trianon Gare de Lyon,Trianon Gare de Lyon,Trianon Gare de Lyon,Trianon Gare de Lyon,...,TRIBE Paris Batignolles,Maxim Folies,The ReMIX Hotel,The ReMIX Hotel,The ReMIX Hotel,Absolute Hotel Paris République,Absolute Hotel Paris République,Absolute Hotel Paris République,The ReMIX Hotel,The ReMIX Hotel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Hotel Taylor,Hotel CIS Paris Maurice Ravel,Hôtel Le Compostelle,Timhotel Paris Gare de Lyon,Motel One Paris-Porte Dorée,Hôtel du Quai de Seine,Hôtel Bellevue et du Chariot d'Or,55 Hôtel Montparnasse,Hôtel Albe Saint Michel,Hotel Le Clos d'Alésia,...,Ibis Styles Paris Gare Saint Lazare,Hotel Splendor Elysées,Louisa Hôtel,Hôtel Albe Saint Michel,Hotel Viator - Gare de Lyon,Simon's Boutique Hôtel - Ice Bar,Novotel Suites Paris Montreuil Vincennes,Hôtel Piapia,Grand Hôtel De Turin,Hotel Jenner
96,Ibis Styles Paris Gare Saint Lazare,Hôtel Vivienne,Hôtel Paris Voltaire,Saphir Grenelle,Agate Hôtel,Régence Etoile,ibis Styles Paris Bercy,Hôtel Du Midi Gare de Lyon,Hotel Agora,Est Hotel,...,B&B Hôtel Paris 17 Batignolles,Hôtel Piapia,Grand Hôtel De Turin,Hotel Viator - Gare de Lyon,Elysees Niel Hotel,Hotel Bellevue Montmartre,HOTEL LUCIEN,Hôtel Botaniste,Hôtel Volney Opéra,Hotel Saint Cyr Etoile
97,Hôtel Paris Voltaire,Hotel Saint Christophe,Hotel Rochechouart,"voco Par

In [72]:
testdfprixT = testdfprix.melt()
testdfprixT = testdfprixT.rename(columns={"variable":"Date", "value":"Prix"})
testdfprixT

,Date,Prix
0,16/11/2021,€ 226
1,16/11/2021,€ 132
2,16/11/2021,€ 196
3,16/11/2021,€ 146
4,16/11/2021,€ 126
...,...,...
4495,30/12/2021,€ 149
4496,30/12/2021,€ 180
4497,30/12/2021,€ 180
4498,30/12/2021,€ 190


In [73]:
testdfhotelT = testdfhotel.melt()
testdfhotelT = testdfhotelT.rename(columns={"variable":"Date", "value":"Hôtel"})
testdfhotelT

,Date,Hôtel
0,16/11/2021,Room Mate Alain - Champs-Elysées
1,16/11/2021,Absolute Hotel Paris République
2,16/11/2021,Hôtel & Spa Saint Jacques
3,16/11/2021,Hotel de l'Aqueduc
4,16/11/2021,Hôtel Hor Les Lumières
...,...,...
4495,30/12/2021,Hotel Jenner
4496,30/12/2021,Hotel Saint Cyr Etoile
4497,30/12/2021,Aparthotel Adagio Paris Nation
4498,30/12/2021,Hôtel des Arts Montmartre


In [123]:
dfprix = pd.concat([testdfprixT, testdfhotelT], axis=1)
dfprix = dfprix.loc[:,~dfprix.columns.duplicated()] # Supprime la deuxième colonne "Date" qui est dupliquée
dfprix

,Date,Prix,Hôtel
0,16/11/2021,€ 226,Room Mate Alain - Champs-Elysées
1,16/11/2021,€ 132,Absolute Hotel Paris République
2,16/11/2021,€ 196,Hôtel & Spa Saint Jacques
3,16/11/2021,€ 146,Hotel de l'Aqueduc
4,16/11/2021,€ 126,Hôtel Hor Les Lumières
...,...,...,...
4495,30/12/2021,€ 149,Hotel Jenner
4496,30/12/2021,€ 180,Hotel Saint Cyr Etoile
4497,30/12/2021,€ 180,Aparthotel Adagio Paris Nation
4498,30/12/2021,€ 190,Hôtel des Arts Montmartre


On peut convertir la date au format "datetime" préconisé par Python, grâce à la fonction "to_datetime".

In [124]:
dfprix['Date'] = pd.to_datetime(testconca['Date'],format='%d/%m/%Y')
dfprix

,Date,Prix,Hôtel
0,2021-11-16,€ 226,Room Mate Alain - Champs-Elysées
1,2021-11-16,€ 132,Absolute Hotel Paris République
2,2021-11-16,€ 196,Hôtel & Spa Saint Jacques
3,2021-11-16,€ 146,Hotel de l'Aqueduc
4,2021-11-16,€ 126,Hôtel Hor Les Lumières
...,...,...,...
4495,2021-12-30,€ 149,Hotel Jenner
4496,2021-12-30,€ 180,Hotel Saint Cyr Etoile
4497,2021-12-30,€ 180,Aparthotel Adagio Paris Nation
4498,2021-12-30,€ 190,Hôtel des Arts Montmartre


Afin d'effectuer des calculs sur la variable "Prix", nous devons transformer le format de cette variable. Pour cela, nous enlevons le caractère "€" ainsi que l'espace qui le suit, puis nous convertirons cette variable au format numérique.

L'instruction ".split()" permet de séparer plusieurs mots d'une chaîne de caractères. On peut donc l'utiliser pour obtenir le nombre de commentaires sous forme de chaîne d'abord, puis pour le transformer en format numérique ensuite. De même pour le prix de la réservation d'hôtels (pour une nuit).

In [131]:
dfprix["Prix"] = dfprix["Prix"].str.replace('[€ ]', '')
dfprix["Prix"] = dfprix["Prix"].str.replace(',', '.')
dfprix["Prix"] = pd.to_numeric(dfprix["Prix"])
dfprix

,Date,Prix,Hôtel
0,2021-11-16,226.0,Room Mate Alain - Champs-Elysées
1,2021-11-16,132.0,Absolute Hotel Paris République
2,2021-11-16,196.0,Hôtel & Spa Saint Jacques
3,2021-11-16,146.0,Hotel de l'Aqueduc
4,2021-11-16,126.0,Hôtel Hor Les Lumières
...,...,...,...
4495,2021-12-30,149.0,Hotel Jenner
4496,2021-12-30,180.0,Hotel Saint Cyr Etoile
4497,2021-12-30,180.0,Aparthotel Adagio Paris Nation
4498,2021-12-30,190.0,Hôtel des Arts Montmartre


In [142]:
dfprix.to_excel('dfprix.xlsx')

On s'est rendu compte en changeant de date de réservation que les hôtels qui s'affichaient n'étaient pas dans le même ordre. Sachant qu'on s'est contenté des 4 premières pages pour chaque jour de réservation, on pouvait s'imaginer que les prix de réservation pour certains hôtels à certains jours ne seraient pas disponibles. La fonction suivante permet de connaître, pour chaque hôtel, à la fois le nombre de jours pour lesquels les prix de réservation figurent dans notre base de données, ainsi que le nombre d'hôtels pour lesquels il y a au moins un prix de réservation répertorié.

In [114]:
dfprix.groupby("Hôtel")["Prix"].count()

Hôtel
55 Hôtel Montparnasse                           33
9Hotel Bastille-Lyon                            27
Aberotel Montparnasse Eiffel                    23
Absolute Hotel Paris République                 43
Acacias Etoile                                   1
                                                ..
ibis Paris Avenue d'Italie 13ème                 8
ibis Paris Gare De L'Est TGV                     1
ibis Paris Gare du Nord Château Landon 10ème     4
ibis Styles Paris Bercy                         32
voco Paris Montparnasse, an IHG Hotel           10
Name: Prix, Length: 253, dtype: int64

In [118]:
np.mean(dfprix.groupby("Hôtel")["Prix"].count())

17.786561264822133

On peut constater que nous disposons d'une base de données avec 253 hôtels, alors qu'idéalement on en aurait eu 100. De plus, en moyenne, nous avons moins de 18 prix répertoriés par hôtel.

La date indiquée est celle du jour de réservation. Autrement dit, pour une réservation la nuit du vendredi à samedi, la date affichée est celle du vendredi. On souhaite créer une variable binaire "Week-end" valant "1" si la réservation est faite pendant le week-end (où l'on s'attend à voir des prix plus élevés) et "0" sinon. De fait, la variable "Week-end" prendra la valeur "1" si la date correspond à un vendredi ou à un samedi, et "0" sinon. On fera de même avec une variable "Vacances" prenant comme valeur "1" si la réservation est faite pendant les vacances et "0" sinon.

In [64]:
testdfprix.to_excel('prix2.xlsx')
testdfhotel.to_excel('hotels2.xlsx')

# 1.2. Web-scraping pour les caractéristiques des hôtels

## 1.2.1. Scraping de la base de données "Hôtels"

Dans cette partie, nous récupérons les données relatives aux caractéristiques des hôtels qui sont directement accessibles sur le moteur de recherche des hôtels (25 hôtels par page) : le nom, la note moyenne, le nombre d'avis de clients, le prix de réservation pour 2 personnes et 1 nuit, et l'arrondissement.

In [143]:
def recup_data(url):
    global hotels,note_moy,nb_avis,prix,arr
    driver = webdriver.Chrome()
    driver.get(url)

    # L'instruction ci-dessous demande à Python de chercher tous les éléments de la classe correspondant aux noms des hôtels, avec une limite de temps afin d'éviter un plantage du code en cas d'échec.

    #print([hotel.text for hotel in wait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "fde444d7ef._c445487e2")))])
    hotels += [hotel.text for hotel in wait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "fde444d7ef._c445487e2")))]
    #print(hotels)

    note_moy += [note.text for note in wait(driver,10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "_9c5f726ff.bd528f9ea6")))]
    #print(note_moy)

    nb_avis += [nbavis.text for nbavis in wait(driver,10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "_4abc4c3d5._1e6021d2f._fb3ba087b._6e869d6e0")))]
    #print(nb_avis)

    # Prix = prix avec promotion.
    prix += [prix.text for prix in wait(driver,10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "fde444d7ef._e885fdc12")))]
    #print(prix)

    arr2 = [arr.text for arr in wait(driver,10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "af1ddfc958.eba89149fb")))]
    arr += [elem for elem in arr2 if elem != 'Indiquer sur la carte']
    #print(arr)

    # Page 1 = https://www.booking.com/searchresults.fr.html?aid=356980&label=gog235jc-1FCAMoTTjjAkgNWANoTYgBAZgBDbgBF8gBDNgBAegBAfgBAogCAagCA7gCw9rQiwbAAgHSAiQ0NDlkNGU5ZC03NzAxLTQ5MzItYjdkYS0wMzVkNDI4NGRiNGXYAgXgAgE&sid=81cd22762f2169ba954d21a297ff0a64&sb=1&src=searchresults&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D356980%3Blabel%3Dgog235jc-1FCAMoTTjjAkgNWANoTYgBAZgBDbgBF8gBDNgBAegBAfgBAogCAagCA7gCw9rQiwbAAgHSAiQ0NDlkNGU5ZC03NzAxLTQ5MzItYjdkYS0wMzVkNDI4NGRiNGXYAgXgAgE%3Bsid%3D81cd22762f2169ba954d21a297ff0a64%3Btmpl%3Dsearchresults%3Bcity%3D-1456928%3Bclass_interval%3D1%3Bdest_id%3D-1456928%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Boffset%3D0%3Bpostcard%3D0%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrpvid%3D227a56f9e7060099%3Bss_all%3D0%3Bssb%3Dempty%3Bsshis%3D0%3Btop_ufis%3D1%26%3B&ss=Paris&is_ski_area=0&ssne=Paris&ssne_untouched=Paris&city=-1456928&checkin_year=2021&checkin_month=11&checkin_monthday=6&checkout_year=2021&checkout_month=11&checkout_monthday=7&group_adults=2&group_children=0&no_rooms=1&sb_changed_dates=1&from_sf=1
    # Page 2 = https://www.booking.com/searchresults.fr.html?label=gog235jc-1FCAMoTTjjAkgNWANoTYgBAZgBDbgBF8gBDNgBAegBAfgBAogCAagCA7gCw9rQiwbAAgHSAiQ0NDlkNGU5ZC03NzAxLTQ5MzItYjdkYS0wMzVkNDI4NGRiNGXYAgXgAgE&sid=81cd22762f2169ba954d21a297ff0a64&aid=356980&sb=1&src=searchresults&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.fr.html%3Faid%3D356980%3Blabel%3Dgog235jc-1FCAMoTTjjAkgNWANoTYgBAZgBDbgBF8gBDNgBAegBAfgBAogCAagCA7gCw9rQiwbAAgHSAiQ0NDlkNGU5ZC03NzAxLTQ5MzItYjdkYS0wMzVkNDI4NGRiNGXYAgXgAgE%3Bsid%3D81cd22762f2169ba954d21a297ff0a64%3Btmpl%3Dsearchresults%3Bcity%3D-1456928%3Bclass_interval%3D1%3Bdest_id%3D-1456928%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Boffset%3D0%3Bpostcard%3D0%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrpvid%3D227a56f9e7060099%3Bss_all%3D0%3Bssb%3Dempty%3Bsshis%3D0%3Btop_ufis%3D1%26%3B&ss=Paris&is_ski_area=0&ssne=Paris&ssne_untouched=Paris&city=-1456928&checkin_year=2021&checkin_month=11&checkin_monthday=6&checkout_year=2021&checkout_month=11&checkout_monthday=7&group_adults=2&group_children=0&no_rooms=1&sb_changed_dates=1&from_sf=1&offset=25

    driver.quit()
    return(hotels, note_moy, nb_avis, arr)

À l'aide d'une boucle, nous pouvons désormais récupérer les données des hôtels pour chaque page du site correspondant aux hôtels parisiens. Attention : cette fonction met plusieurs minutes pour s'exécuter. Afin de faciliter l'avancée du projet, nous nous proposons de sauvegarder le dataframe dans un fichier Excel que l'on pourra directement charger pour les étapes suivantes.

In [11]:
hotels = []
note_moy = []
nb_avis = []
arr = []
prix = []

for i in range(40):
    recup_data(liste[i])

Enfin, nous pouvons synthétiser l'ensemble des données obtenues dans un dataframe. Ce format nous permettra d'exploiter plus facilement ces données.

In [33]:
dfhotels = pd.DataFrame({"Hôtels":hotels,"Note moyenne":note_moy,"Nombre de commentaires":nb_avis,"Prix":prix,"Arrondissement":arr})
dfhotels

,Hôtels,Note moyenne,Nombre de commentaires,Prix,Arrondissement
0,Vice Versa,"7,9",573 expériences vécues,€ 126,"15e arr., Paris"
1,Le Robinet d'Or,"9,0",646 expériences vécues,€ 184,"10e arr., Paris"
2,Hôtel Gaston,"8,2",375 expériences vécues,€ 111,"17e arr., Paris"
3,Absolute Hotel Paris République,"7,9",2 519 expériences vécues,€ 136,"11e arr., Paris"
4,The ReMIX Hotel,"8,4",1 116 expériences vécues,€ 99,"19e arr., Paris"
...,...,...,...,...,...
995,Monsieur Saintonge,"8,4",506 expériences vécues,€ 253,"3e arr., Paris"
996,Monsieur George Hotel & Spa - Champs-Elysées,"9,1",202 expériences vécues,€ 511,"8e arr., Paris"
997,Maison Albar Hotels Le Champs-Elysées,"8,5",433 expériences vécues,€ 328,"17e arr., Paris"
998,Room Mate Alain - Champs-Elysées,"8,6",1 173 expériences vécues,€ 211,"16e arr., Paris"


Afin de pouvoir exploiter ce dataframe, nous souhaitons supprimer les éventuels doublons et valeurs manquantes.

In [34]:
dfhotels = dfhotels.drop_duplicates()
dfhotels = dfhotels.dropna()
dfhotels

In [45]:
dfhotels

,Hôtels,Note moyenne,Nombre de commentaires,Prix,Arrondissement
0,Vice Versa,"7,9",573 expériences vécues,€ 126,"15e arr., Paris"
1,Le Robinet d'Or,"9,0",646 expériences vécues,€ 184,"10e arr., Paris"
2,Hôtel Gaston,"8,2",375 expériences vécues,€ 111,"17e arr., Paris"
3,Absolute Hotel Paris République,"7,9",2 519 expériences vécues,€ 136,"11e arr., Paris"
4,The ReMIX Hotel,"8,4",1 116 expériences vécues,€ 99,"19e arr., Paris"
...,...,...,...,...,...
994,Le Burgundy Paris,"9,1",581 expériences vécues,€ 528,"1er arr., Paris"
995,Monsieur Saintonge,"8,4",506 expériences vécues,€ 253,"3e arr., Paris"
996,Monsieur George Hotel & Spa - Champs-Elysées,"9,1",202 expériences vécues,€ 511,"8e arr., Paris"
997,Maison Albar Hotels Le Champs-Elysées,"8,5",433 expériences vécues,€ 328,"17e arr., Paris"


Nous nous rendons compte qu'il y avait plus de 200 doublons parmi les hôtels. Maintenons, nous sauvegardons le dataframe sous forme de fichier Excel. Cela nous permet de retrouver nos données scrappées sans avoir à réexécuter l'ensemble du code.

In [48]:
dfhotels.to_excel('dfhotels.xlsx')

(Si on est amené à modifier la base de données, n'exécuter que les lignes ci-dessous)

In [9]:
dfhotels = pd.read_excel('dfhotels.xlsx', index_col='Unnamed: 0')
dfhotels.head()

,Hôtels,Note moyenne,Nombre de commentaires,Prix,Arrondissement
0,Vice Versa,"7,9",573 expériences vécues,€ 126,"15e arr., Paris"
1,Le Robinet d'Or,"9,0",646 expériences vécues,€ 184,"10e arr., Paris"
2,Hôtel Gaston,"8,2",375 expériences vécues,€ 111,"17e arr., Paris"
3,Absolute Hotel Paris République,"7,9",2 519 expériences vécues,€ 136,"11e arr., Paris"
4,The ReMIX Hotel,"8,4",1 116 expériences vécues,€ 99,"19e arr., Paris"


On cherche à rajouter à ces données le nombre d'étoiles de chaque hôtel. Cette information ne peut pas être scrapée facilement à partir du site Booking.com, car les étoiles s'affichent sous forme d'images. On essaye donc de récupérer l'information à partir d'un script, qui consiste à :
1) chercher sur Google le nom de l'hôtel ;

2) récupérer le nombre d'étoiles sur la page.

In [55]:
from selenium.common.exceptions import NoSuchElementException

In [180]:
def recup_etoiles(hotels):
    etoiles = []
    driver = webdriver.Chrome()
    url = 'https://www.google.com/search?q=étoiles hotel '
    driver.get(url)
    for hotel in hotels:
        driver.get(url+str(hotel))
        # Test 1 : regarder partout sur la page si le nombre d'étoiles apparaît sous forme de texte
        if len(driver.find_elements(By.XPATH, "//*[contains(text(),'5 étoiles')]"))!=0:
            etoiles.append(5)
        elif len(driver.find_elements(By.XPATH, "//*[contains(text(),'4 étoiles')]"))!=0:
            etoiles.append(4)
        elif len(driver.find_elements(By.XPATH, "//*[contains(text(),'3 étoiles')]"))!=0:
            etoiles.append(3)
        elif len(driver.find_elements(By.XPATH, "//*[contains(text(),'2 étoiles')]"))!=0:
            etoiles.append(2)
        elif len(driver.find_elements(By.XPATH, "//*[contains(text(),'1 étoile')]"))!=0:
            etoiles.append(1)
        else:
            # Test 1 : regarder si le bandeau de Google (à droite) affiche le nombre d'étoiles
            try:
                texte = driver.find_element(By.CLASS_NAME, "YhemCb").text
                if '5' in texte:
                    etoiles.append(5)
                elif '4' in texte:
                    etoiles.append(4)
                elif '3' in texte:
                    etoiles.append(3)
                elif '2' in texte:
                    etoiles.append(2)
                elif '1' in texte:
                    etoiles.append(1)
            # Sinon, le nombre d'étoiles n'apparaît pas sous forme de chaîne de caractères simple. On affiche alors "NA".
            except NoSuchElementException:
                etoiles.append(np.nan)
    return etoiles

Dans les faits, Google nous empêche d'effectuer trop de recherches d'affilée. Sinon, le moteur de recherche demande s'il a affaire à un robot ou non, et nous ne pouvons alors pas accéder au résultat de la recherche. On décide donc d'appliquer la fonction à notre base de données en plusieurs temps.

In [178]:
etoiles1 = recup_etoiles(dfhotels['Hôtels'][0:200])
len(etoiles1)

On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la boucle
On entre dans la

200

In [192]:
etoiles2 = recup_etoiles(dfhotels['Hôtels'][200:400])
len(etoiles2)

200

In [181]:
etoiles3 = recup_etoiles(dfhotels['Hôtels'][400:600])
etoiles3[:5]

[5, 3, 3, 5, 5]

In [186]:
etoiles4 = recup_etoiles(dfhotels['Hôtels'][600:len(dfhotels['Hôtels'])+1])
etoiles4[:5]

[3, 3, 4, 1, 4]

In [191]:
len(etoiles1)

200

In [193]:
dfhotels['Etoiles']=etoiles1+etoiles2+etoiles3+etoiles4
dfhotels.head()

,Hôtels,Note moyenne,Nombre de commentaires,Prix,Arrondissement,Etoiles
0,Vice Versa,"7,9",573 expériences vécues,€ 126,"15e arr., Paris",4.0
1,Le Robinet d'Or,"9,0",646 expériences vécues,€ 184,"10e arr., Paris",3.0
2,Hôtel Gaston,"8,2",375 expériences vécues,€ 111,"17e arr., Paris",NaN
3,Absolute Hotel Paris République,"7,9",2 519 expériences vécues,€ 136,"11e arr., Paris",NaN
4,The ReMIX Hotel,"8,4",1 116 expériences vécues,€ 99,"19e arr., Paris",4.0


In [194]:
dfhotels = dfhotels.dropna()
len(dfhotels)

740

In [195]:
dfhotels.to_excel('dfhotels.xlsx')

## 1.2.2. Scraping de la base de données "Notes"

On utilise des fonctions similaires à celles créées précédemment. Nous allons créer une boucle sur les url de la liste href_hotels afin de scraper pour chaque hôtel le détail de ses notes ainsi que son nom, et obtenir ainsi un dataframe concaténable à dfhotels.

In [ ]:
def recup_href(page):
    href_hotel=[]
    liste = cree_liste(page)
    for i in range(40):
    #On choisit ici le nombre de pages de résultats exploitées ; il y a 25 hôtels par pages 
        driver = webdriver.Chrome(ChromeDriverManager().install())
        #driver = webdriver.Chrome()
        driver.get(liste[i])
        href_hotel+=[my_elem.get_attribute("href") for my_elem in WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "h3._23bf57b84 > a")))]           
        driver.quit()
    return(href_hotel)

In [ ]:
page = 'https://www.booking.com/searchresults.fr.html?label=gog235jc-1FCAMoTTjjAkgNWANoTYgBAZgBDbgBF8gBDNgBAegBAfgBAogCAagCA7gCw9rQiwbAAgHSAiQ0NDlkNGU5ZC03NzAxLTQ5MzItYjdkYS0wMzVkNDI4NGRiNGXYAgXgAgE&sid=29099b70bfcd7cf703ca2b9b7570611a&aid=356980&sb=1&sb_lp=1&src=theme_landing_index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Fhotel%2Findex.fr.html%3Faid%3D356980%3Blabel%3Dgog235jc-1FCAMoTTjjAkgNWANoTYgBAZgBDbgBF8gBDNgBAegBAfgBAogCAagCA7gCw9rQiwbAAgHSAiQ0NDlkNGU5ZC03NzAxLTQ5MzItYjdkYS0wMzVkNDI4NGRiNGXYAgXgAgE%3Bsid%3D29099b70bfcd7cf703ca2b9b7570611a%3Bsrpvid%3D96be43728e9d017d%26%3B&ss=Paris%2C+%C3%8Ele-de-France%2C+France&is_ski_area=&checkin_year=2022&checkin_month=1&checkin_monthday=10&checkout_year=2022&checkout_month=1&checkout_monthday=11&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=paris&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1456928&dest_type=city&iata=PAR&place_id_lat=48.85668&place_id_lon=2.351476&search_pageview_id=813443bdfdfb00e8&search_selected=true&search_pageview_id=813443bdfdfb00e8&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0'
href_hotels=recup_href(page)
print(href_hotels[:3])

Tout d'abord, on construit des fonctions de mise en forme des données obtenues par selenium.

In [2]:
def mef_nom(nom):
    x=""
    for lettre in nom:
        x+=lettre
    return x[6:]

mef_nom(['H', 'ô', 't', 'e', 'l', ' ', 'H', 'o', 't', 'e', 'l', ' ', 'E', 'l', 'y', 's', 'e', 'e', 's', ' ', 'O', 'p', 'e', 'r', 'a'])

'Hotel Elysees Opera'

In [3]:
def mef_notes(notes):
    y=[]
    for note in notes:
        y.append(float(note.replace(',' , '.')))
    return y

mef_notes(['9,4', '8,1', '8,6', '8,6', '8,2', '8,9', '8,3'])

[9.4, 8.1, 8.6, 8.6, 8.2, 8.9, 8.3]

In [4]:
def recup_data(href):
    hotel=[]
    #dans l'ordre d'apparition sur la page
    perso=[]
    equip=[]
    prop=[]
    conf=[]
    rap_qp=[]
    situ=[]
    wifi=[]
    
    for url in href :
        driver = webdriver.Chrome(ChromeDriverManager().install())
        #driver = webdriver.Chrome()
        driver.get(url)
        nom=WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//div[@class='hp__hotel-title']/h2"))).text
        notes=[elem.get_attribute("innerHTML") for elem in driver.find_elements_by_xpath("//div[@class='c-score-bar']//span[@class='c-score-bar__score']")]
        #la liste des notes est dans l'ordre : perso,equip... 
        
        nom=mef_nom(nom)
        notes=mef_notes(notes)
        
        #Ajout aux listes globales 
        hotel.append(nom)
        perso.append(notes[0])
        equip.append(notes[1])
        prop.append(notes[2])
        conf.append(notes[3])
        rap_qp.append(notes[4])
        situ.append(notes[5])
        wifi.append(notes[6])  

        driver.quit()
    
    df = pd.DataFrame({'hotel':hotel,'perso':perso,'equip':equip,'prop':prop,'conf':conf,'rap_qp':rap_qp,'situ':situ,'wifi':wifi})
    
    return df

In [ ]:
df1=recup_data(href_hotels)
df1.to_excel('notes_hotels.xlsx')

## 1.2.3. Associer les bases de données "Hôtels" et "Notes"

Pour pouvoir unifier deux bases de données selon une même caractéristique (ici, le nom des hôtels), il faut que cette caractéristique soit présente dans chaque base de données sous la forme d'une colonne, et que cette colonne ait le même nom dans les deux bases. Ce n'est actuellement pas le cas.

In [196]:
notes_hotels = pd.read_excel('notes_hotels.xlsx', index_col='Unnamed: 0')
notes_hotels

,hotel,perso,equip,prop,conf,rap_qp,situ,wifi
0,Hôtel R de Paris - Boutique Hotel,9.0,9.5,9.5,9.3,8.9,8.7,9.1
1,Le 7 Eiffel Hotel,8.9,8.3,8.5,8.3,7.9,7.5,9.0
2,Petit Madeleine Hôtel,8.5,9.2,9.1,8.2,8.7,8.4,8.8
3,Hôtel Des Marronniers,9.1,9.7,8.8,8.2,8.5,8.1,9.1
4,Hotel des Nations Saint Germain,8.8,9.1,7.9,7.8,7.8,7.6,7.7
...,...,...,...,...,...,...,...,...
995,'hôtel\nAMHotel Italie,8.5,8.5,8.5,7.9,8.0,7.7,8.3
996,'hôtel\nAdagio Paris 19ème Cité de la Musique,7.4,8.4,8.1,8.1,7.9,7.6,6.8
997,Le A,9.6,9.6,9.3,9.3,8.8,8.3,8.7
998,Mercure Paris 19 Philharmonie La Villette,8.2,8.2,7.6,7.7,7.2,7.0,7.6


In [198]:
notes_hotels = notes_hotels.rename(columns={'hotel':'Hôtels'})
notes_hotels

,Hôtels,perso,equip,prop,conf,rap_qp,situ,wifi
0,Hôtel R de Paris - Boutique Hotel,9.0,9.5,9.5,9.3,8.9,8.7,9.1
1,Le 7 Eiffel Hotel,8.9,8.3,8.5,8.3,7.9,7.5,9.0
2,Petit Madeleine Hôtel,8.5,9.2,9.1,8.2,8.7,8.4,8.8
3,Hôtel Des Marronniers,9.1,9.7,8.8,8.2,8.5,8.1,9.1
4,Hotel des Nations Saint Germain,8.8,9.1,7.9,7.8,7.8,7.6,7.7
...,...,...,...,...,...,...,...,...
995,'hôtel\nAMHotel Italie,8.5,8.5,8.5,7.9,8.0,7.7,8.3
996,'hôtel\nAdagio Paris 19ème Cité de la Musique,7.4,8.4,8.1,8.1,7.9,7.6,6.8
997,Le A,9.6,9.6,9.3,9.3,8.8,8.3,8.7
998,Mercure Paris 19 Philharmonie La Villette,8.2,8.2,7.6,7.7,7.2,7.0,7.6


In [200]:
dfhotels_merged = pd.merge(dfhotels, notes_hotels, on='Hôtels')
dfhotels_merged.head()

,Hôtels,Note moyenne,Nombre de commentaires,Prix,Arrondissement,Etoiles,perso,equip,prop,conf,rap_qp,situ,wifi
0,Vice Versa,"7,9",573 expériences vécues,€ 126,"15e arr., Paris",4.0,8.1,8.7,8.3,8.1,7.9,7.5,8.8
1,Vice Versa,"7,9",573 expériences vécues,€ 126,"15e arr., Paris",4.0,8.1,8.7,8.3,8.1,7.9,7.5,8.8
2,The ReMIX Hotel,"8,4",1 116 expériences vécues,€ 99,"19e arr., Paris",4.0,7.8,8.8,8.8,8.8,8.4,8.3,8.3
3,TRIBE Paris Batignolles,"9,1",176 expériences vécues,€ 130,"17e arr., Paris",5.0,8.8,9.6,9.4,9.4,9.2,8.8,9.0
4,Hotel Aida Marais,"7,9",1 046 expériences vécues,€ 141,"10e arr., Paris",3.0,8.6,8.7,8.3,7.9,8.1,7.7,7.9


Il semble qu'il y ait encore des "duplicates". On va les supprimer.

In [201]:
dfhotels_merged = dfhotels_merged.drop_duplicates()
dfhotels_merged.head()

,Hôtels,Note moyenne,Nombre de commentaires,Prix,Arrondissement,Etoiles,perso,equip,prop,conf,rap_qp,situ,wifi
0,Vice Versa,"7,9",573 expériences vécues,€ 126,"15e arr., Paris",4.0,8.1,8.7,8.3,8.1,7.9,7.5,8.8
2,The ReMIX Hotel,"8,4",1 116 expériences vécues,€ 99,"19e arr., Paris",4.0,7.8,8.8,8.8,8.8,8.4,8.3,8.3
3,TRIBE Paris Batignolles,"9,1",176 expériences vécues,€ 130,"17e arr., Paris",5.0,8.8,9.6,9.4,9.4,9.2,8.8,9.0
4,Hotel Aida Marais,"7,9",1 046 expériences vécues,€ 141,"10e arr., Paris",3.0,8.6,8.7,8.3,7.9,8.1,7.7,7.9
5,Hôtel Hor Les Lumières,"8,6",972 expériences vécues,€ 117,"11e arr., Paris",4.0,9.4,8.8,9.0,8.3,8.5,8.5,8.5


Comme dans la base de données "Prix", les données ne sont pas exploitables telles quelles. On constate en effet que les colonnes "Note moyenne", "Nombre de commentaires", "Prix" et "Arrondissement" ne sont pas au format numérique. Avant de passer au format numérique, il va nous falloir enlever les chaînes de caractères, et donc convertir les colonnes au format 'str'.

In [204]:
dfhotels_merged.dtypes

Hôtels                     object
Note moyenne               object
Nombre de commentaires     object
Prix                       object
Arrondissement             object
Etoiles                   float64
perso                     float64
equip                     float64
prop                      float64
conf                      float64
rap_qp                    float64
situ                      float64
wifi                      float64
dtype: object

In [208]:
dfhotels_merged["Note moyenne"] = dfhotels_merged["Note moyenne"].astype(str)
dfhotels_merged["Note moyenne"] = dfhotels_merged["Note moyenne"].str.replace(',', '.')
dfhotels_merged["Note moyenne"] = pd.to_numeric(dfhotels_merged["Note moyenne"])

dfhotels_merged["Prix"] = dfhotels_merged["Prix"].astype(str)
dfhotels_merged["Prix"] = dfhotels_merged["Prix"].str.replace('€ ', '')
dfhotels_merged["Prix"] = pd.to_numeric(dfhotels_merged["Prix"])

dfhotels_merged["Arrondissement"] = dfhotels_merged["Arrondissement"].astype(str)
dfhotels_merged["Arrondissement"] = dfhotels_merged["Arrondissement"].str.replace('e arr., Paris', '')
dfhotels_merged["Arrondissement"] = dfhotels_merged["Arrondissement"].str.replace('er arr., Paris', '')
dfhotels_merged["Arrondissement"] = pd.to_numeric(dfhotels_merged["Arrondissement"])

dfhotels_merged["Nombre de commentaires"] = dfhotels_merged["Nombre de commentaires"].astype(str)
dfhotels_merged["Nombre de commentaires"] = dfhotels_merged["Nombre de commentaires"].str.replace(' expériences vécues', '')
dfhotels_merged["Nombre de commentaires"] = dfhotels_merged["Nombre de commentaires"].str.replace(' ', '')
dfhotels_merged["Nombre de commentaires"] = pd.to_numeric(dfhotels_merged["Nombre de commentaires"])

<ipython-input-208-c17e51ff78a0>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  dfhotels_merged["Arrondissement"] = dfhotels_merged["Arrondissement"].str.replace('e arr., Paris', '')
<ipython-input-208-c17e51ff78a0>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  dfhotels_merged["Arrondissement"] = dfhotels_merged["Arrondissement"].str.replace('er arr., Paris', '')


In [209]:
dfhotels_merged.head()

,Hôtels,Note moyenne,Nombre de commentaires,Prix,Arrondissement,Etoiles,perso,equip,prop,conf,rap_qp,situ,wifi
0,Vice Versa,7.9,573,126,15,4.0,8.1,8.7,8.3,8.1,7.9,7.5,8.8
2,The ReMIX Hotel,8.4,1116,99,19,4.0,7.8,8.8,8.8,8.8,8.4,8.3,8.3
3,TRIBE Paris Batignolles,9.1,176,130,17,5.0,8.8,9.6,9.4,9.4,9.2,8.8,9.0
4,Hotel Aida Marais,7.9,1046,141,10,3.0,8.6,8.7,8.3,7.9,8.1,7.7,7.9
5,Hôtel Hor Les Lumières,8.6,972,117,11,4.0,9.4,8.8,9.0,8.3,8.5,8.5,8.5


In [210]:
dfhotels_merged.to_excel('dfhotels_merged.xlsx', index=False)

Sous cette forme, les données sont exploitables. L'analyse des deux bases est effectuée dans les deux autres fichiers du repository.